## 1. FGSM-nontargeted

In [1]:
# import the necessary packages
from tensorflow.keras.losses import MSE
import tensorflow as tf

def generate_image_adversary(model, image, label, eps=2 / 255.0):
	# cast the image
	image = tf.cast(image, tf.float32)
 
 	# record our gradients
	with tf.GradientTape() as tape:
		# explicitly indicate that our image should be tacked for
		# gradient updates
		tape.watch(image)
  
		# use our model to make predictions on the input image and
		# then compute the loss
		pred = model(image)
		loss = MSE(label, pred)
  
  	# calculate the gradients of loss with respect to the image, then
	# compute the sign of the gradient
	gradient = tape.gradient(loss, image)
	signedGrad = tf.sign(gradient)
 
	# construct the image adversary
	adversary = (image + (signedGrad * eps)).numpy()
 
	# return the image adversary to the calling function
	return adversary

In [2]:
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [3]:
ClassNames = ['Aircraft Carrier', 'Bulkers', 'Car Carrier', 'Container Ship', 'Cruise', 'DDG', 'Recreational', 'Sailboat', 'Submarine', 'Tug']

In [5]:
import os
import cv2
import numpy as np
base_path = r'C:\dev\workspace\Asia_Arrived_La\Data\Ships_dataset\train'
data_image = os.path.join(base_path, 'images')
data_label = os.path.join(base_path, 'labels')
traindata = []
pathi = os.path.join(data_image)
pathl = os.path.join(data_label)

for file in os.listdir(pathi):
    file_p  = os.path.join(pathi, file)
    image = cv2.imread(file_p)
    image = np.array(image)
    image=cv2.resize(image, (64,64),interpolation=cv2.INTER_LINEAR)
    traindata.append(image)

In [6]:
import os
trainlabel = []
for file in os.listdir(pathl):
    file_p  = os.path.join(pathl, file)
    f = open(file_p, "r")
    a = int(f.read(1))
    trainlabel.append(a)

## 2. model

In [11]:
from tensorflow.keras import datasets,layers,models
model = models.Sequential()
model.add(layers.Conv2D(64,(3,3),activation='relu',input_shape=(64,64,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

C:\Users\User\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
for i in np.random.choice(np.arange(0, len(traindata)), size=(10,)):
    image = traindata[i]
    label = trainlabel[i]

    # 이미지에 대한 적대적 샘플 생성
    adversary = generate_image_adversary(model, image.reshape(1, 64, 64, 3), label, eps=0.1)
    pred = model.predict(adversary)
    
    # 예측된 숫자 레이블을 클래스 이름으로 변환
    imagePred = ClassNames[label]
    adversaryPred = ClassNames[pred[0].argmax()]

    # 확신도 계산
    confidence = pred[0].max() * 100  # 확률을 백분율로 변환
    color = (0, 255, 0)

    # 예측이 다를 경우 빨간색으로 표시
    if imagePred != adversaryPred:
        color = (0, 0, 255)

    # 이미지를 [0, 255] 범위로 스케일 조정
    adversary = adversary.reshape((64, 64, 3)) * 255
    adversary = np.clip(adversary, 0, 255).astype("uint8")
    image = (image * 255).astype("uint8")

    # 이미지 및 적대적 이미지에 클래스 이름과 확신도 표시
    image = cv2.resize(image, (384, 384))
    adversary = cv2.resize(adversary, (384, 384))

    cv2.putText(image, f"{imagePred}", (2, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.95, (0, 255, 0), 2)
    cv2.putText(adversary, f"{adversaryPred} ({confidence:.2f}%)", (2, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.95, color, 2)

    output = np.hstack([image, adversary])
    cv2.imshow("FGSM Adversarial Images", output)
    cv2.waitKey(0)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step


In [ ]:
import torch 
import numpy as np

extended_image = []
extended_labels = []

for i in range(len(traindata)):
    image = traindata[i]
    label = trainlabel[i]
    extended_image.append(image)
    extended_label.append(label)
    adversary = generate_image_adversary(model, image.reshape(1, 64, 64, 3), label, eps=0.1)
    adversary = adversary.reshape(64,64,3)
    extended_image.append(adversary)
    extended_label.append(label)

extended_image = np.array(extended_image)
extended_label = np.array(extended_label)

extended_image = torch.tensor(extended_image).permute(0,3,1,2).float()
extended_label = torch.tensor(extended_label).long()

extended_dataset = torch.utils.data.TensorDataset(extended_image, extended_labels)
extended_loader = torch.utils.data.DataLoader(extended_dataset, batch_size=64, shuffle=True)